In [3]:
in_file =r'../../data/clean/articles.csv' 

import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine


In [11]:
articles=pd.read_csv(in_file,index_col=False)
articles.CR.fillna('',inplace=True)
df_ref=articles.CR
refs_dict={}
i=0
for ref in df_ref:
    refs_dict[i]=[ref for ref in ref.split('; ')]
    i=i+1

In [12]:
cite = open('cite_all.csv', 'w')
cite.write('citing,cited\n')

for row in refs_dict:
    for ref in refs_dict[row]:
        cite.write(str(row)+','+ ref.replace(',','')+'\n')
cite.close()

In [4]:
cite_all=pd.read_csv('cite_all.csv')
cite_all.fillna('',inplace=True)

In [ ]:
dg=cite_all.groupby('cited').count()
dg1=dg[dg.citing==1]
dg=dg[dg.citing>1]
dg.reset_index(inplace=True)

In [ ]:
cite_all['cited_id']=-1
i=0
dg_cited=dg.cited.values
for x in cite_all.cited:
    try:
        ind=dg[dg.cited==x].index[0]
        cite_all.set_value(i,'cited_id',ind)
    except:
        pass
    i=i+1
    if i % 1000 ==0:
        print i,
print

In [11]:
cite_all=cite_all[cite_all.cited_id>-1]

In [1]:
df=cite_all[['citing','cited_id']]
df.to_csv('cite_all_id.csv')

NameError: name 'cite_all' is not defined

In [22]:
df=pd.read_csv('cite_all_id.csv',index_col='Unnamed: 0')

In [23]:
x=df.citing.max()
y=df.cited_id.max()
mm=np.zeros((x+1,y+1))
mm.shape

(1207, 11622)

In [24]:
for row in df.iterrows():
    mm[row[1][0],row[1][1]]=1

In [25]:
#bib=np.triu(np.dot(mm,mm.transpose()),1)
bib = 1-pairwise_distances(mm, metric="cosine")
bib = np.round(np.triu(bib,1),3)
bib

array([[ 0.   ,  0.112,  0.   , ...,  0.015,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       ..., 
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ]])

In [26]:
biblio = open('bib.csv', 'w')
biblio.write('source,target,weight,type\n')

for row in range(bib.shape[0]):
    for col in range(row,bib.shape[1]):
        if bib[row][col] > .1:
            biblio.write(str(row)+','+str(col)+','+str(bib[row][col])+',undirected\n')
biblio.close()

In [27]:
df=pd.read_csv('bib.csv')
df[df.weight>.7]

,source,target,weight,type
1676,129,137,0.802,undirected
2798,212,618,0.816,undirected
4096,341,1195,0.918,undirected
5889,552,680,0.735,undirected
7787,937,1012,0.705,undirected
